# Import libraries

In [1]:
import os
import pickle
import sys
import tweepy
import requests
from dotenv import load_dotenv
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

sys.path.append('../../')
import config

load_dotenv(os.path.join(config.PROJ_PATH, '.env')) 

True

# Create Twitter client

In [2]:
client = tweepy.Client(bearer_token=os.environ["BEARER_TOKEN"])

In [9]:
# Set search query
query = "vaca"

# Searching for Tweets from the last 7 days

In [13]:
tweets = client.search_recent_tweets(query=query, tweet_fields=['context_annotations', 'created_at'], max_results=10)

for tweet in tweets.data:
    print(tweet.text)
    if len(tweet.context_annotations) > 0:
        print(tweet.context_annotations)

RT @jdoedoe101101: @Kicillofok Pero ya tienen Vaca Muerta, hace 4 años que controlan YPF... y tuvieron que pedir otros 45.000 millones de d…
[{'domain': {'id': '10', 'name': 'Person', 'description': 'Named people in the world like Nelson Mandela'}, 'entity': {'id': '1179511859944148992', 'name': 'Axel Kicillof'}}, {'domain': {'id': '35', 'name': 'Politician', 'description': 'Politicians in the world, like Joe Biden'}, 'entity': {'id': '1179511859944148992', 'name': 'Axel Kicillof'}}, {'domain': {'id': '131', 'name': 'Unified Twitter Taxonomy', 'description': 'A taxonomy of user interests. '}, 'entity': {'id': '847878884917886977', 'name': 'Politics', 'description': 'Politics'}}, {'domain': {'id': '131', 'name': 'Unified Twitter Taxonomy', 'description': 'A taxonomy of user interests. '}, 'entity': {'id': '1070032753834438656', 'name': 'Political figures', 'description': 'Politician'}}, {'domain': {'id': '131', 'name': 'Unified Twitter Taxonomy', 'description': 'A taxonomy of user inter

# Searching for Tweets from the full-archive of public Tweets

In [ ]:
# tweets = client.search_all_tweets(query=query, tweet_fields=['context_annotations', 'created_at'], max_results=10)

# for tweet in tweets.data:
#     print(tweet.text)
#     if len(tweet.context_annotations) > 0:
#         print(tweet.context_annotations)

# Getting more than 100 Tweets at a time using paginator

In [ ]:
# Replace the limit=1000 with the maximum number of Tweets you want
for tweet in tweepy.Paginator(client.search_recent_tweets, query=query,
                              tweet_fields=['context_annotations', 'created_at'], max_results=10).flatten(limit=10):
    print(tweet.id)
    print(tweet)

# Make search query

In [5]:
search_keywords = ['mal amada', 'vaca', 'burra']
complementary_words = []
places = []
lang = 'pt'

query = ''

if len(search_keywords) > 0:
    keywords_query = ''
    for idx, keyword in enumerate(search_keywords):
        if idx == 0:
            keywords_query += '('
        if idx == len(search_keywords) - 1:
            keywords_query += f'"{keyword}")'
            break
        keywords_query += f'"{keyword}" OR '
    query += f'{keywords_query} '

if len(complementary_words) > 0:
    comp_words_query = ''
    for idx, word in enumerate(complementary_words):
        if idx == 0:
            comp_words_query += '('
        if idx == len(complementary_words) - 1:
            comp_words_query += f'"{word}")'
            break
        comp_words_query += f'"{word}" '
    query += f'{comp_words_query} '

if len(places) > 0:
    places_query = ''
    for idx, place in enumerate(places):
        if idx == 0:
            places_query += '('
        if idx == len(places) - 1:
            places_query += f'place:"{place}")'
            break
        places_query += f'place:"{place}" '
    query += f'{places_query} '


query += f'lang:{lang} -is:retweet'

In [6]:
tweets = client.search_recent_tweets(query=query, 
                                     tweet_fields=['id', 'text', 'author_id', 'geo', 'lang','created_at'], 
                                     user_fields=['id', 'name', 'username', 'profile_image_url','created_at'], 
                                     place_fields=['id', 'country', 'country_code', 'place_type', 'name', 'geo'],
                                     expansions=['author_id', 'geo.place_id'],
                                     max_results=10)

# Get users informations from tweets
users = {u["id"]: u for u in tweets.includes['users']}
# Get places informations from tweets, if exists
try:
    places = {p["id"]: p for p in tweets.includes['places']}
except KeyError:
    print("Tweets extraídos não tem componente de localização.")

Tweets extraídos não tem componente de localização.


In [8]:
for tweet in tweets.data:
    tweet_data = {}
    extracted_keywords = [keyword for keyword in search_keywords if keyword in tweet.text]
    tweet_data['keywords_extraction'] = extracted_keywords
    tweet_data['id'] = int(tweet.id)
    tweet_data['text'] = tweet.text
    tweet_data['lang'] = tweet.lang
    tweet_data['created_at'] = tweet.created_at
    if users[tweet.author_id]:
        user = users[tweet.author_id]
        tweet_data['user_id'] = int(user.id)
        tweet_data['user_name'] = user.name
        tweet_data['user_screen_name'] = user.username
        tweet_data['user_profile_image_url'] = user.profile_image_url
        tweet_data['user_created_at'] = user.created_at
    try:
        place = places[tweet.geo['place_id']]
        tweet_data['place_id'] = int(place.id)
        tweet_data['country'] = place.country
        tweet_data['country_code'] = place.country_code
        tweet_data['place_type'] = place.place_type
        tweet_data['place_name'] = place.name
        tweet_data['coordinates'] = place.geo['bbox']

    except:
        pass

    print(tweet_data)
    break
    

{'keywords_extraction': [], 'id': 1700626720880050342, 'text': 'Mds n posso beber ipa quer eu fico BURRA', 'lang': 'pt', 'created_at': datetime.datetime(2023, 9, 9, 21, 46, 21, tzinfo=datetime.timezone.utc), 'user_id': 189309268, 'user_name': 'Pudin', 'user_screen_name': 'whycophine', 'user_profile_image_url': 'https://pbs.twimg.com/profile_images/1690556497258795008/OREw1yIR_normal.jpg', 'user_created_at': datetime.datetime(2010, 9, 10, 22, 57, 26, tzinfo=datetime.timezone.utc)}
